# Script to Download Sourcer Pipeline Results from DynamoDB and Display on Streamlit

In [11]:
# Add to your existing imports
import boto3  # type: ignore
import json
from decimal import Decimal
from datetime import datetime
from typing import Any, List
from botocore.exceptions import ClientError # type: ignore

# Initialize DynamoDB client
# Create session with default profile
session = boto3.Session(profile_name='root')  # or your profile name

dynamodb = session.resource('dynamodb')
dynamodb_client = session.client('dynamodb')
table_name = "vc-sourcing-analysis"

In [14]:
def query_top_repos_by_score(table_name: str = "vc-sourcing-analysis", limit: int = 10) -> List[dict[str, Any]]:
    """Query top repositories by score from DynamoDB."""
    try:
        table = dynamodb.Table(table_name)
        
        # Scan table and sort by score (since we can't easily query by score range)
        response = table.scan()
        
        items = response['Items']
        
        # Sort by final_score in descending order
        sorted_items = sorted(items, key=lambda x: float(x.get('final_score', 0)), reverse=True)
        
        return sorted_items[:limit]
        
    except Exception as e:
        print(f"❌ Error querying DynamoDB: {e}")
        return []

def print_dynamodb_results(items: List[dict[str, Any]]) -> None:
    """Print formatted results from DynamoDB query."""
    print("\n🏆 TOP REPOSITORIES FROM DYNAMODB:")
    print("=" * 60)
    
    for i, item in enumerate(items, 1):
        repo_name = item.get('repo_name', 'Unknown')
        score = float(item.get('final_score', 0))
        analysis_date = item.get('analysis_date', 'Unknown')
        
        # Get OSS Insight data
        oss_data = item.get('oss_insight_data', {})
        total_score = oss_data.get('total_score', 'N/A')
        stars = oss_data.get('stars', 'N/A')
        description = oss_data.get('description', 'No description available')
        
        print(f"\n## {repo_name} | 📊 Score: {score}")
        print(f"📈 Total Score: {total_score} | ⭐ Stars: {stars}")
        print(f"📝 Description: {description}")
        
        # Show repo analysis details if available
        if item.get('repo_analysis'):
            repo_analysis = item['repo_analysis']
            problem_solved = repo_analysis.get('problem_solved', 'Problem description not available')
            print(f"📖 Problem Solved: {problem_solved}")
            print(f"🎯 Problem Statement Clarity: {repo_analysis.get('problem_clarity_score', 'N/A')}/5")
            print(f"⚡ Ease of Adoption: {repo_analysis.get('adoption_ease_score', 'N/A')}/5")
            print(f"💊 Project Maturity & Community Health: {repo_analysis.get('maturity_health_score', 'N/A')}/5")
        
        # Show community sentiment details if available
        if item.get('community_analysis'):
            community = item['community_analysis']
            print(f"😊 Community Sentiment: {community.get('excitement_score', 'N/A')}/5")
            print(f"📈 Problem Solution Fit: {community.get('problem_solution_fit_score', 'N/A')}/5")
            print(f"💊 Credibility & Adoption: {community.get('credibility_adoption_score', 'N/A')}/5")
            
            # Add quotes if available
            key_praise = community.get('key_praise_quote')
            main_criticism = community.get('main_criticism')
            
            if key_praise:
                print(f"💬 Key Praise Quote: {key_praise}")
            if main_criticism:
                print(f"⚠️ Main Criticism: {main_criticism}")

print("✅ Updated DynamoDB results display function")

✅ Updated DynamoDB results display function


In [15]:
# Query and display top results from DynamoDB
print(f"\n🔍 Querying top results from DynamoDB...")
top_repos = query_top_repos_by_score(table_name, limit=5)
print_dynamodb_results(top_repos)


🔍 Querying top results from DynamoDB...

🏆 TOP REPOSITORIES FROM DYNAMODB:

## n8n-io/n8n | 📊 Score: 93.02
📈 Total Score: 34714.0041 | ⭐ Stars: 7482
📝 Description: Fair-code workflow automation platform with native AI capabilities. Combine visual building with custom code, self-host or cloud, 400+ integrations.
📖 Problem Solved: It removes the friction of building and managing complex integrations by providing a self-hostable, code-extensible no-code platform with hundreds of prebuilt connectors.
🎯 Problem Statement Clarity: 5/5
⚡ Ease of Adoption: 5/5
💊 Project Maturity & Community Health: 5/5
😊 Community Sentiment: 4/5
📈 Problem Solution Fit: 5/5
💊 Credibility & Adoption: 4/5
💬 Key Praise Quote: It took 20 minutes to have a workflow which runs every hour that calls Miniflux to get my RSS feed data, Mealie to get my recipes, and then upload those files to Dropbox—complete with logging and monitoring out of the box.
⚠️ Main Criticism: The source-available license’s commercial-use rest